# Picard iteration on iota to reach zero current
This example runs `GVEC` from a python script multiple times with different parameters and restarting from the final state of each previous run.

The new iota profile that excludes the current contribution is computed from the previous state, fitted to a polynomial and prescribed via the parameterfile to the next run. 

Along the Picard iteration, radial resolution and maximum iteration count is increased, as well.

**Note:** This script needs to `import gvec`, please run it in a virtual environment of python, where the python module `gvec` is already installed!


In [ ]:
import os
import time
from pathlib import Path
import contextlib
import shutil

import numpy as np
import matplotlib.pyplot as plt

# needs `pip install` of gvec in virtual environment, and to be run in that environment!!!
os.environ["OMP_NUM_THREADS"] = "4"
import gvec  # using run & modifying the parameters & postprocessing

In [ ]:
@contextlib.contextmanager
def chdir(path: Path | str):
    """
    Contextmanager to change the current working directory.

    Using a context has the benefit of automatically changing back to the original directory when the context is exited, even if an exception is raised.
    """
    path = Path(path)
    old_dir = Path(os.getcwd())

    os.chdir(path)
    yield
    os.chdir(old_dir)

In [ ]:
# total number of gvec runs
iterations = 12

template = "parameter.ini"
rundir = "rungvec"
outfile_suffix = "_x"

# degree of the polynomial to fit iota-iota_curr
iota_poly_degree = 9
# number of elements during the Picard iterations (gvec_nelems + (gvec_nelems_max-gvec_nelems)*(2*(i//2)/iterations)**2)
gvec_nelems = 2
gvec_nelems_max = 10  # 40
# maximum number of iterations in each GVEC run, min(gvec_maxiter,gvec_miniter* 2**i)
gvec_miniter = 10  # 100
gvec_maxiter = 100  # 1000

initLA_at_restart = "F"

In [ ]:
print(f"Toplevel working directory: {os.getcwd()}")
path = Path(rundir)
if path.exists():
    print(f"Removing existing run directory {path}")
    shutil.rmtree(path)
if not path.exists():
    path.mkdir()
    print(f"created run directory {path}")

for i in range(0, iterations + 1):
    start_time = time.time()
    print(f"Iteration {i}")
    params = {}
    restartfile = ""
    # find previous state
    if i > 0:
        previous_path = Path(f"{rundir}/{i-1:02d}")
        statefile = sorted(previous_path.glob("*State*.dat"))[-1]
        print(f"Using statefile {statefile}")
        restartfile = Path("../../") / statefile

        with gvec.State(previous_path / "parameter.ini", statefile) as state:
            rho = np.sqrt(np.linspace(0, 1, 41)[1:])
            ev = gvec.Evaluations(rho=rho, theta="int", zeta="int", state=state)
            state.compute(ev, "iota", "iota_curr", "N_FP")
            # get polynomial fit of iota - iota_curr
            iota_coefs = np.polyfit(
                rho**2, ev["iota"] - ev["iota_curr"], iota_poly_degree
            )
            iota_curr_rms = np.sqrt(
                (ev.iota_curr**2).mean("rad").item()
            )  # possible early stop condition
            print(f"iota_curr_rms: {iota_curr_rms:.3e}")
            # print(f"New iota parameters: {iota_coefs[::-1]}")

            # set new parameters
            params["init_LA"] = initLA_at_restart
            params["sign_iota"] = 1
            # currently adapt_parameter_file expects strings for iota coefficients
            params["iota_coefs"] = "(/" + ", ".join(map(str, iota_coefs[::-1])) + "/)"

    params["maxiter"] = int(np.amin([gvec_maxiter, gvec_miniter * 2**i]))
    params["sgrid_nElems"] = int(
        gvec_nelems + (gvec_nelems_max - gvec_nelems) * (2 * (i // 2) / iterations) ** 2
    )
    print("-" * 40)
    print(f"Running iteration {i}/{iterations}")

    # prepare the run directory
    path = Path(f"{rundir}/{i:02d}")
    if path.exists():
        print(f"Removing existing run directory {path}")
        shutil.rmtree(path)
    if not path.exists():
        path.mkdir()
        print(f"created run directory {path}")

    # copy parameterfile with modifications
    gvec.util.adapt_parameter_file(template, path / "parameter.ini", **params)
    # run gvec
    with chdir(f"{rundir}/{i:02d}"):
        gvec.run("parameter.ini", restartfile, stdout_path="stdout.txt")
    end_time = time.time()
    print(f"iteration took {end_time-start_time : 5.1f} seconds.")
    print("=" * 40)

print("Picard iteration finished")
print("=" * 40)

In [ ]:
# post-process
wmhd = np.zeros(iterations + 1)
iota_curr_rms = np.zeros(iterations + 1)
profiles = {}
# extract profiles for visualization
for i in range(0, iterations + 1):
    profiles[i] = {}
    run_path = Path(f"{rundir}/{i:02d}")
    statefile = sorted(run_path.glob("*State*.dat"))[-1]
    with gvec.State(run_path / "parameter.ini", statefile) as state:
        rho = np.sqrt(np.linspace(0, 1, 41)[1:])
        ev = gvec.Evaluations(rho=rho, theta="int", zeta="int", state=state)
        state.compute(ev, "iota", "iota_curr", "N_FP")
        ev_volint = gvec.Evaluations(rho="int", theta="int", zeta="int", state=state)
        state.compute(ev_volint, "W_MHD")
        # print(f"W_MHD={ev_volint.W_MHD:.7e}")
        profiles[i]["ev"] = ev[["rho", "iota", "iota_curr", "I_tor"]]
        wmhd[i] = ev_volint["W_MHD"]
        iota_curr_rms[i] = np.sqrt((ev.iota_curr**2).mean("rad").item())

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 3))
axs[1].plot(wmhd)
axs[1].set(xlabel="picard iterations for curr. constraint", ylabel="total MHD energy")
axs[0].semilogy(iota_curr_rms)
axs[0].set(
    xlabel="picard iterations for curr. constraint",
    ylabel="root mean square of (iota_curr)",
)

plt.savefig(f"wmhd_{outfile_suffix}.pdf", format="pdf", bbox_inches="tight")

In [ ]:
maxiter_plot = iterations

# === Plot evolution of profiles === #
fig, axs = plt.subplots(1, 3, figsize=(15, 5), tight_layout=True, sharex=True)


for var, ax in zip(["I_tor", "iota_curr"], [axs[2], axs[1]]):
    for step in range(0, maxiter_plot + 1):
        col = "blue" if (step == maxiter_plot) else "black"
        alp = 1.0 if (step == maxiter_plot) else 0.2 + 0.3 * (step / maxiter_plot)
        lsty = "-x" if (step == maxiter_plot) else "-"
        pev = profiles[step]["ev"]
        ax.semilogy(
            pev.rho**2, np.abs(pev[var].values) + 1.0e-16, lsty, color=col, alpha=alp
        )
    ax.set(
        title=pev[var].attrs["long_name"],
        xlabel="$\\rho^2$",
        # ylabel=f"$|{ev[var].attrs['symbol']}|$",
    )

sign_iota = -1
for ax in [axs[0]]:
    for step in range(0, maxiter_plot + 1, 1):
        col = "blue" if (step == maxiter_plot) else "black"
        alp = 1.0 if (step == maxiter_plot) else 0.2 + 0.3 * (step / maxiter_plot)
        lsty = "-x" if (step == maxiter_plot) else "-"
        pev = profiles[step]["ev"]
        ax.plot(pev.rho**2, sign_iota * pev["iota"], lsty, color=col, alpha=alp)
        coefs = np.polyfit(pev.rho**2, pev["iota"] - pev["iota_curr"], iota_poly_degree)
        y_fit = np.polyval(coefs, pev.rho**2)
        print(
            f"step={step},fiterr={np.amax(np.abs(pev['iota'].values-pev['iota_curr'].values-y_fit)):.3e}"
        )

        # ax.plot(ev.rho**2,sign_iota*y_fit,"--",color="red",alpha=0.1+0.9*step/iterations)
    ax.set(
        title=pev["iota"].attrs["long_name"],
        xlabel="$\\rho^2$",
    )


axs[2].set(ylabel=r"$|I_\text{tor}|\quad [A]$")
axs[1].set(ylabel=r"$|\iota_\text{curr}|$")
axs[0].set(ylabel=r"$\iota$")
axs[2].set_ylim(1e-3, 1e6)
axs[1].set_ylim(1e-9, 1)
axs[0].set_ylim(0.75, 1.05)

plt.savefig(
    f"current_convergence{outfile_suffix}.pdf", format="pdf", bbox_inches="tight"
)

In [ ]:
# last state
run_path = Path(f"{rundir}/{iterations:02d}")
statefile = sorted(run_path.glob("*State*.dat"))[-1]
with gvec.State(run_path / "parameter.ini", statefile) as state:
    rho1d = np.linspace(0.0, 1, 21)
    theta1d = np.linspace(0.0, 2 * np.pi, 41)
    zeta1d = np.linspace(0, 2 * np.pi / 5, 12, endpoint=False)
    ev = gvec.Evaluations(rho=rho1d, theta=theta1d, zeta=zeta1d, state=state)
    state.compute(ev, "X1", "X2", "LA", "N_FP")

# === Cross-sections === #
fig, axs = plt.subplots(
    3, 4, figsize=(15, 6), tight_layout=True, sharex=True, sharey=True
)

cuts = np.linspace(0, 2 * np.pi / ev.N_FP.item(), 12)
rho_levels_vis = np.linspace(0, 1 - 1e-10, 5)
theta_levels_vis = np.linspace(0, 2 * np.pi, 8 + 1)
for ax, zeta in zip(axs.flat, cuts):
    ds = ev.sel(zeta=zeta, method="nearest")
    R_vis = ds.X1[:, :]
    Z_vis = ds.X2[:, :]
    rho_vis = ds.X1[:, :] * 0 + ev.rho
    theta_vis = ds.X1[:, :] * 0 + ev.theta
    thetastar_vis = ds.LA[:, :] + ev.theta
    ax.contour(R_vis, Z_vis, rho_vis, rho_levels_vis, colors="blue")
    ax.contour(R_vis, Z_vis, thetastar_vis, theta_levels_vis, colors="red")

    ax.set(
        aspect="equal",
        title=f"$\\zeta/\\pi = {zeta/np.pi:.2f}$",
    )

for ax in axs[-1, :]:
    ax.set_xlabel(f"${ev.X1.attrs['symbol']}$")
for ax in axs[:, 0]:
    ax.set_ylabel(f"${ev.X2.attrs['symbol']}$")

plt.savefig(
    f"final_cross-sections{outfile_suffix}.pdf", format="pdf", bbox_inches="tight"
)